In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd

import analyze

/Users/steve/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [7]:
state_data = pd.read_csv("states.csv")

In [16]:
state_data.head()

,date,state,positive,negative,pending,hospitalized,death,total,dateChecked
0,20200323,AK,22.0,946.0,NaN,0.0,NaN,968,2020-03-23T20:00:00Z
1,20200323,AL,167.0,1665.0,NaN,NaN,0.0,1832,2020-03-23T20:00:00Z
2,20200323,AR,174.0,906.0,0.0,13.0,0.0,1080,2020-03-23T20:00:00Z
3,20200323,AS,NaN,NaN,NaN,NaN,0.0,0,2020-03-23T20:00:00Z
4,20200323,AZ,265.0,309.0,6.0,NaN,2.0,580,2020-03-23T20:00:00Z


In [89]:
single_state_cases = state_data[state_data.state.eq("NY")].sort_values("date")
single_state_cases = single_state_cases[single_state_cases.total.notna()]

In [90]:
days = single_state_cases.date.to_numpy() - 20200000
cases = single_state_cases.total.to_numpy()

In [91]:
print(days)
print(cases)

[304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321
 322 323]
[   78   122   361   404   197   234   265   216   216  3200  3303  5272
  5493  7206 14597 22284 32427 45437 61401 78289]


In [92]:
model = analyze.ExponentialGrowthRateEstimator()
model.fit(day=days, cases=cases)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   19
Model:                            GLM   Df Residuals:                       17
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                   -inf
Date:                Mon, 23 Mar 2020   Deviance:                       35193.
Time:                        23:45:40   Pearson chi2:                 1.61e+04
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -94.6699      0.389   -243.336      0.000     -95.432     -93.907
x1             0.3240      0.001    266.868      0.000       0.322       0.326
==============================================================================
"""

In [93]:
print(model.growth_rate())
print(model.growth_rate_confint())

0.38265827529873775
(0.37937199085944484, 0.38595238914553476)


This is to see which states have a substantial amount of death data. The `(date, count)` column contains the number of days where the number of deaths is > 10.

In [43]:
state_data[state_data.death.notna() & state_data.death.ge(10)].groupby(['state']).agg(['count']).sort_values(('date', 'count'), ascending=False)

,date,positive,negative,pending,hospitalized,death,total,dateChecked
,count,count,count,count,count,count,count,count
state,,,,,,,,
WA,13,13,13,0,0,13,13,13
CA,7,7,7,1,0,7,7,7
NY,6,6,6,0,3,6,6,6
GA,5,5,5,0,0,5,5,5
FL,4,4,4,4,3,4,4,4
LA,4,4,4,0,0,4,4,4
NJ,4,4,4,4,0,4,4,4
CT,1,1,1,0,1,1,1,1


Similarly, we can look at the number of days where the hospitalizations is not NA.

In [96]:
state_data[state_data.hospitalized.notna()].groupby(['state']).agg(['count']).sort_values(('date', 'count'), ascending=False)

,date,positive,negative,pending,hospitalized,death,total,dateChecked
,count,count,count,count,count,count,count,count
state,,,,,,,,
AK,3,3,3,0,3,0,3,3
MA,3,3,3,0,3,3,3,3
VA,3,3,3,0,3,3,3,3
OK,3,3,3,3,3,3,3,3
OH,3,3,3,0,3,3,3,3
NY,3,3,3,0,3,3,3,3
ND,3,3,3,0,3,3,3,3
WV,3,3,3,3,3,3,3,3
